![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Revisiting Machine Learning Case Study

- In this lab, you will use `learningSet.csv` file which we used during the class. 

### Instructions

Complete the following steps on the categorical columns in the dataset:

- Check for null values in all the columns
- Create a new empty list called `drop_list`. We will append to this list a set of columns to be droped later. Add the following columns to this:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP` - we are including state already
- Identify columns that over 85% missing values and add them to the previous list.
- Remove the columns included in the `drop_list` from the dataframe
- Now, reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```

In [1]:
import pandas as pd

import functions

In [2]:
# Load data
data = pd.read_csv('../learningSet.csv')

/var/folders/zg/vcnq21zn4cv4pv2ft4lqfk6r0000gn/T/ipykernel_2221/2285174264.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../learningSet.csv')


In [3]:
# Format column names
data_formatted = functions.format_column_names(data)

In [4]:
# Create DataFrame with % of null values per column
nulls_percent_df = pd.DataFrame(data_formatted.isna().sum()/len(data_formatted)).reset_index() 

# Rename columns
nulls_percent_df.columns = ['column_name', 'nulls_percentage']

# Sort column according to % of null values
nulls_percent_df.sort_values(['nulls_percentage'], ascending=False)

column_name  nulls_percentage
414     rdate_5          0.999906
436     ramnt_5          0.999906
412     rdate_3          0.997464
434     ramnt_3          0.997464
413     rdate_4          0.997055
..          ...               ...
168       ethc3          0.000000
167       ethc2          0.000000
166       ethc1          0.000000
165       hhd12          0.000000
240       tpe11          0.000000

[481 rows x 2 columns]

In [5]:
# Create a function to get columns with null values over x %

def get_null_columns(df: pd.DataFrame, threshold: float = 0.25) -> list:
    '''
    This function takes in a pandas DataFrame and returns a list with the 
    columns with a percentage value of null values above the input threshold.

    Input:
        df -> pandas DataFrame
        threshold -> float, percentage value threshold (i.e 0.25)

    Output:
        list -> columns with % null values above threshold
    '''
    
    # Get percentage of null values for each column
    # Apply reset_index to move column names to a column to convert to DataFrame to be able to apply filters
    nulls_percent_df = pd.DataFrame(df.isna().sum()/len(df)).reset_index() 

    # Rename columns
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']

    # Filter columns above threshold
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]

    # Convert to list
    drop_columns_list = list(columns_above_threshold['column_name'])

    return drop_columns_list



In [6]:
# Create list of columns to drop
drop_list = []

# Add 'OSOURCE' and 'ZIP' 
drop_list += ['osource', 'zip']

In [7]:
# Identify columns that over 85% missing values
cols_with_nulls_over_85 = get_null_columns(data_formatted, 0.85)

# Add them to the previous list
drop_list += cols_with_nulls_over_85

In [8]:
# Remove the columns included in the drop_list from the dataframe
data_formatted.drop(columns=drop_list, inplace=True)

In [9]:
# Replace NAs with 'F'
data_formatted['gender'] = data_formatted['gender'].fillna('F')

In [10]:
# Show unique values for gender column
data_formatted['gender'].unique()

array(['F', 'M', ' ', 'C', 'U', 'J', 'A'], dtype=object)

In [11]:
# Replace all values except 'M', 'F' with 'other'
data_formatted['gender'] = [ 'other' if x in [' ', 'C', 'U', 'J', 'A'] else x for x in data_formatted['gender']]

In [12]:
# Show value counts of gender
data_formatted['gender'].value_counts()

gender
F        51277
M        39094
other     5041
Name: count, dtype: int64